# Run Bank Marketing Inference Job via Python SDK
This notebook demonstrates how to submit and monitor the command job that scores the Bank Marketing AutoML model on a compute cluster.

In [69]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Job
from azure.ai.ml import MLClient, load_job
import os
from dotenv import load_dotenv
load_dotenv()

# Initialize MLClient
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id=os.getenv("AZURE_SUBSCRIPTION_ID"),
    resource_group_name=os.getenv("AZURE_RESOURCE_GROUP"),
    workspace_name=os.getenv("AZURE_WORKSPACE_NAME")
)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [ ]:
model_name = "best automl model" #example "azureml_calm_lion_57btkgfdw9_3_output_mlflow_log_model_1598679586"
model_version = "1"

In [43]:
ml_model = ml_client.models.get(model_name, model_version)  # Example to verify connection

In [44]:
ml_client.models.download(name=model_name, version=model_version, download_path="./model")

AzureCliCredential.get_token_info failed: Failed to invoke the Azure CLI



In [ ]:
# Replace the old cell content with this

import shutil
from pathlib import Path

# Source folder produced by model download
src_dir = Path(f"./model/{model_name}/mlflow-model")
# Destination folder you want to use
dst_dir = Path("./src/model/mlflow-model")

if not src_dir.exists():
    raise FileNotFoundError(f"Expected source directory not found: {src_dir.resolve()}")

# If you want to completely replace any previous copy:
if dst_dir.exists():
    # Remove the existing destination to ensure a clean copy
    shutil.rmtree(dst_dir)

# Copy the entire MLflow model directory
shutil.copytree(src_dir, dst_dir)

print(f"Copied MLflow model directory to {dst_dir.resolve()}")
print(f"Contents: {[p.name for p in dst_dir.iterdir()]}")


Copied MLflow model directory to C:\repo\azureml-examples\sdk\python\jobs\automl-standalone-jobs\automl-classification-task-bankmarketing\cluster-inference\model\mlflow-model
Contents: ['conda.yaml', 'MLmodel', 'model.pkl', 'python_env.yaml', 'requirements.txt']


In [50]:

#remove the model folder
shutil.rmtree(f"./model/{model_name}")

In [ ]:
#copy environment from model directory to environment.yml

import shutil

shutil.copy(f"./src/model/mlflow-model/conda.yaml", "./environment.yml")


'./environment.yml'

In [77]:
# Load job definition from job.yaml
job = load_job("job.yaml")

In [78]:
# Submit the job and wait for completion
submitted_job = ml_client.jobs.create_or_update(job)
ml_client.jobs.stream(submitted_job.name)

Uploading src (0.97 MBs): 100%|##########| 971659/971659 [00:01<00:00, 826608.58it/s]


pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFileJobOutput'> and will be ignored


RunId: sad_camera_q4h2smdrpg
Web View: https://ml.azure.com/runs/sad_camera_q4h2smdrpg?wsid=/subscriptions/6025ba02-1dfd-407f-b358-88f811c7c7aa/resourcegroups/sc1-ml1/workspaces/sc1ml1

Execution Summary
RunId: sad_camera_q4h2smdrpg
Web View: https://ml.azure.com/runs/sad_camera_q4h2smdrpg?wsid=/subscriptions/6025ba02-1dfd-407f-b358-88f811c7c7aa/resourcegroups/sc1-ml1/workspaces/sc1ml1



In [ ]:
# Access the predictions output URI
out_uri = submitted_job.outputs.predictions
print(f"Predictions saved to: {out_uri}")